In [ ]:
import open3d as o3d
import numpy as np
import copy
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from winsound import Beep
import time
import glob

In [ ]:
def draw_point_clouds(point_clouds, black_point_cloud = None):
    pc_copy = []
    colors = [[1, 0.706, 0], [0, 0.651, 0.929], [0.3, 0.351, 0.529]]
    for idx, x in enumerate(point_clouds):
        temp = copy.deepcopy(x)
        if idx < len(colors):
            temp.paint_uniform_color(colors[idx])
        else:
            temp.paint_uniform_color(np.random.rand(3))
        pc_copy.append(temp)
    
    if black_point_cloud is not None:
        temp = copy.deepcopy(black_point_cloud)
        temp.paint_uniform_color([0,0,0])
        pc_copy.append(temp)
   
    o3d.visualization.draw_geometries(pc_copy)

In [ ]:
frames_to_interpolate = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\converted\pcd_compressed\\"
frames_to_interpolate = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\cropped_frames\\"
frames_to_interpolate = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\rotated_and_translated\\"
frames_to_interpolate = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Data\Data\01_scene_01_omar\01_lidar\01_vehicle_lidar_robosense\vehicle_lidar_robosense_driving_direction_east\s110_first_east\matched\\"
frames_to_interpolate = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\new_try\pcd_without_nan\\"
filenames_to_interpolate = sorted(glob.glob(frames_to_interpolate + "166*.pcd"))
interpolated_poses_path = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\rotated_and_translated\100000000\interpolation\interpolated_poses_tum.txt"

target_path = r"C:\Users\amoff\Documents\Meine Textdokumente\Masterarbeit\Daten\converted\pcd\0_0_intersection_dirty2_xyzi_down_transformed_compressed_ground_removed.pcd"
target = o3d.io.read_point_cloud(target_path)

frames = []
for file in filenames_to_interpolate:
    frames.append(o3d.io.read_point_cloud(file))
with open(interpolated_poses_path) as f:
    lines = f.read().splitlines()

In [ ]:
draw_point_clouds([frames[10]])

In [ ]:
for frame, pose in zip(frames, lines):
    t, x, y, z, qx, qy, qz, qw = [float(i) for i in pose.split(" ")]
    frame_copy = copy.deepcopy(frame).translate((x, y, z))
    R = frame_copy.get_rotation_matrix_from_quaternion((qx, qy, qz, qw))
    frame_copy.rotate(R)
    draw_point_clouds([target, frame_copy])
    break

In [ ]:
for x, pose in zip(frames, lines):
    print(len(pose))
    print((pose))
    break

In [ ]:
source = copy.deepcopy(frames[1])
source.translate([5,5,0])



vis = o3d.visualization.Visualizer()
vis.create_window()

# target.transform(flip_transform)
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(source)
vis.add_geometry(target)
threshold = 0.05
icp_iteration = 100

for i in range(icp_iteration):
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source, target, threshold, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
    source.transform(reg_p2l.transformation)
    vis.update_geometry(source)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(1)    # Pause 5.5 seconds
vis.destroy_window()
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Info)